In [19]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
pip install seaborn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [23]:
import pandas as pd
import mysql.connector
import os

# List of CSV files and their corresponding table names
csv_files = [
    ('customers.csv', 'customers'),
    ('orders.csv', 'orders'),
    ('sellers.csv', 'sellers'),
    ('products.csv', 'products'),
    ('geolocation.csv', 'geolocation'),
    ('payments.csv', 'payments'),
    ('order_items.CSV', 'order_items')# Added payments.csv for specific handling
]

# Connect to the MySQL database
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='258025',
    database='ecommerce'
)
cur = conn.cursor()


# Folder containing the CSV files
folder_path = 'C:/Users/ASUS/Music/E commerce project , WS cube'

def get_sql_type(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return 'INT'
    elif pd.api.types.is_float_dtype(dtype):
        return 'FLOAT'
    elif pd.api.types.is_bool_dtype(dtype):
        return 'BOOLEAN'
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return 'DATETIME'
    else:
        return 'TEXT'

for csv_file, table_name in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    
    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(file_path)
    
    # Replace NaN with None to handle SQL NULL
    df = df.where(pd.notnull(df), None)
    
    # Debugging: Check for NaN values
    print(f"Processing {csv_file}")
    print(f"NaN values before replacement:\n{df.isnull().sum()}\n")

    # Clean column names
    df.columns = [col.replace(' ', '_').replace('-', '_').replace('.', '_') for col in df.columns]

    # Generate the CREATE TABLE statement with appropriate data types
    columns = ', '.join([f'`{col}` {get_sql_type(df[col].dtype)}' for col in df.columns])
    create_table_query = f'CREATE TABLE IF NOT EXISTS `{table_name}` ({columns})'
    cursor.execute(create_table_query)

    # Insert DataFrame data into the MySQL table
    for _, row in df.iterrows():
        # Convert row to tuple and handle NaN/None explicitly
        values = tuple(None if pd.isna(x) else x for x in row)
        sql = f"INSERT INTO `{table_name}` ({', '.join(['`' + col + '`' for col in df.columns])}) VALUES ({', '.join(['%s'] * len(row))})"
        cursor.execute(sql, values)

    # Commit the transaction for the current CSV file
    conn.commit()

# Close the connection
conn.close()

Processing customers.csv
NaN values before replacement:
customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64



NameError: name 'cursor' is not defined

In [24]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import mysql.connector 

db = mysql.connector.connect(host = "localhost",
                            username = "root",
                            password = "258025",
                            database = "ecommerce")

cur = db.cursor()


In [ ]:
#1. List all unique cities where customers are located.
query = """select distinct(customer_city) from customers"""
cur.execute ( query )
data = cur.fetchall()
data

In [ ]:
#2 Count the number of orders placed in 2017.
q = """select count(order_id)from orders where order_purchase_timestamp = 2017"""
cur.execute(q)
data = cur.fetchall()
"Total orders placed in 2017 are ", data[0][0]



#3 Find the total sales per category.

In [ ]:
q = """ select upper(p.product_category), round(sum(py.payment_value),2) as Total_sale
from products as p
join order_items
on p.product_id = order_items.product_id
join payments as py
on order_items.order_id = py.order_id
group by product_category
order by Total_sale desc;
 """
cur.execute(q)
data = cur.fetchall()
df= pd.DataFrame(data, columns = ["Category","Sale"]) 
df

#3 Calculate the percentage of orders that were paid in installments

In [ ]:
q = """ select sum((case when payment_installments >=1 then 1 else 0 end)) / count(*) *100 from payments"""
cur.execute(q)
data = cur.fetchall()
'Total percentage of orders that were paid in installments is ', data

#4 Count the number of customers from each state. 

In [ ]:
q = """select count(customer_id) as Total_customer, customer_state
from customers
group by customer_state
order by Total_customer desc"""

cur.execute(q)
data = cur.fetchall()
df = pd.DataFrame(data, columns = ["Total_customer" , "State"])
plt.bar(df["State"], df["Total_customer"])
plt.xticks(rotation = 90)
plt.show()

Intermediate Queries-
1. Calculate the number of orders per month in 2018.

In [ ]:
q = """select 
monthname(order_purchase_timestamp) as month , count(order_id) as T_order
from orders
where order_purchase_timestamp = 2018
group by month
order by month desc """
cur.execute(q)
data = cur.fetchall()
df = pd.DataFrame(data, columns = ["month", "T_order"])
df

#2Find the average number of products per order, grouped by customer city.

In [ ]:
cur = cursor   

In [ ]:
q = """with count_per_order as (
select orders.order_id, orders.customer_id ,count(order_items.order_id) as oc
from orders join order_items 
on orders.order_id = order_items.order_id
group by orders.order_id, orders.customer_id
)
select customers.customer_city, round(avg(oc),2) as Avg_order
from customers join count_per_order
on customers.customer_id = count_per_order.customer_id
group by customers.customer_city
order by  Avg_order desc
limit 10; """
cur.execute(q)
data = cur.fetchall()
df = pd.DataFrame(data, columns = ["City", "Avg_order"])
df 

3. Calculate the percentage of total revenue contributed by each product category.

In [ ]:
q = """ select upper(p.product_category),
 round((sum(py.payment_value)/(select sum(payment_value) from payments))*100,2) as Total_sale
from products as p
join order_items
on p.product_id = order_items.product_id
join payments as py
on order_items.order_id = py.order_id
group by product_category
order by Total_sale desc
limit 10; """
cur.execute(q)
data = cur.fetchall()
df = pd.DataFrame(data, columns = ["Revenue", "Cateogory(in %)"])
df 

4. Identify the correlation between product price and the number of times a product has been purchased.

In [ ]:
q = """ select products.product_category,
count(order_items.product_id),
round(avg(order_items.price),2)
from products join order_items
on products.product_id = order_items.product_id
group by products.product_category; """
cur.execute(q)
data = cur.fetchall()
df = pd.DataFrame(data, columns = ["product", "Purchased", "Avg_price"])
a = df["Purchased"]
b = df["Avg_price"]
np.corrcoef([a,b])


5. Calculate the total revenue generated by each seller, and rank them by revenue.

In [ ]:
q = """  with revenue as (select sum(p.payment_value)as total_revenue, s.seller_id from payments as p
join order_items on p.order_id = order_items.order_id join sellers as s
on order_items.seller_id = s.seller_id
group by s.seller_id)

select seller_id,total_revenue,
rank()over(order by total_revenue DESC ) as rank_by_revenue
from revenue
limit 10;

  """
cur.execute(q)
data = cur.fetchall()
df = pd.DataFrame(data, columns = ["Seller_Id", "Total_Revenue","Ranks"])
df 

In [ ]:
cur = cursor

# Advance query
1. Calculate the moving average of order values for each customer over their order history.

In [ ]:
q = """  SELECT customer_id, 
       order_purchase_timestamp,
       AVG(payment) OVER (
           PARTITION BY customer_id 
           ORDER BY order_purchase_timestamp 
           ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
       ) AS mov_avg
FROM (
    SELECT o.customer_id, 
           o.order_purchase_timestamp, 
           p.payment_value AS payment 
    FROM payments p 
    JOIN orders o ON p.order_id = o.order_id
) sub;

  """
cur.execute(q)
data = cur.fetchall()
df = pd.DataFrame(data, columns = ["Seller_Id", "Total_Revenue","Ranks"])
df 

In [ ]:
cur = conn.cursor()


2. Calculate the cumulative sales per month for each year.

In [ ]:
q = """  select years , months, payment, sum(payment)
over (order by years, months ) cummulative_sales from 
(select year(orders.order_purchase_timestamp) as years,
month(orders.order_purchase_timestamp) as months,
round(sum(payments.payment_value),2) as payment from orders join payments on 
orders.order_id = payments.order_id
group by years , months 
order by years , months 
) as a;
  """
cur.execute(q)
data = cur.fetchall()
df = pd.DataFrame(data, columns = ["Years" , "Months","Payment","Cummulative Payment  "])
df 

3. Calculate the year-over-year growth rate of total sales.

In [ ]:
q = """  with a as (select year(orders.order_purchase_timestamp) as years, round(sum(payments.payment_value),2) as Payment 
from orders join payments on
orders.order_id = payments.order_id
group by years
order by years)

SELECT years,Payment, LAG(Payment) OVER (ORDER BY years) AS previous_year_sales,
    ROUND(((Payment - LAG(Payment) OVER (ORDER BY years)) / 
            LAG(Payment) OVER (ORDER BY years)) * 100, 2) AS growth_percentage
            FROM a
ORDER BY years;
 ;
  """
cur.execute(q)
data = cur.fetchall()
df = pd.DataFrame(data, columns = ["Years" ,"payment","Previous Year sale","growth rate"])
df 

4. Calculate the retention rate of customers, defined as the percentage of customers who make another purchase within 6 months of their first purchase.

In [ ]:
q = """   With a as (select customers.customer_id , date(min(orders.order_purchase_timestamp)) as First_order
from customers join orders on 
customers.customer_id = orders.customer_id
group by customer_id)

select a.customer_id, count(distinct orders.order_purchase_timestamp) from a
join orders on a.customer_id = orders.customer_id 
and orders.order_purchase_timestamp > first_order and
orders.order_purchase_timestamp < date_add(first_order, interval 6 month)
group by a.customer_id;
  """
cur.execute(q)
data = cur.fetchall()
df = pd.DataFrame(data, columns = ["Customer_id","Repeated customer "]) #unfortunately no repeated customer for 6 Months 
df 

#5 Identify the top 3 customers who spent the most money in each year.

In [ ]:
q = """    SELECT years, customer_id, payment, d_rank 
FROM (
    SELECT 
        YEAR(o.order_purchase_timestamp) AS years,
        o.customer_id, 
        SUM(p.payment_value) AS payment,
        DENSE_RANK() OVER (
            PARTITION BY YEAR(o.order_purchase_timestamp) 
            ORDER BY SUM(p.payment_value) DESC
        ) AS d_rank
    FROM orders o
    JOIN payments p ON p.order_id = o.order_id
    GROUP BY YEAR(o.order_purchase_timestamp), o.customer_id
) AS a
WHERE d_rank <= 3;
  """
cur.execute(q)
data = cur.fetchall()
df = pd.DataFrame(data, columns = ["Year","Customer id ", "Payments", "Rank"])  
df 